In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import dotenv
from pathlib import Path

env_file = "../.env"

if os.path.exists(env_file):
    dotenv.load_dotenv(env_file, verbose=True)
    print("Loaded environment variables from .env file.")

cwd = os.getcwd()
# for some reason appending to PATH you need it to be string
sys.path.append(str(Path(cwd).parent / "src"))
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

# from research_tools.gpu import get_gpus_available
# os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in get_gpus_available()])

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

Loaded environment variables from .env file.


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict, Optional
import torch
from research_tools.utils import set_seed

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device == torch.device("cuda")

model_id = "LLM-LAT/zephyr7b-beta-rmu-lat-unlearn-wmdp-bio-cyber"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "meta-llama/Meta-Llama-3-8B"
model_id = "HuggingFaceH4/zephyr-7b-beta"

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, trust_remote_code=True, token=hf_access_token
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/mnt/align4_drive/data/huggingface/hub/models--HuggingFaceH4--zephyr-7b-beta/.no_exist/892b3d7a7b1cf10c7a701c60881cd93df615734c/adapter_config.json'
Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/mnt/align4_drive/data/huggingface/hub/models--HuggingFaceH4--zephyr-7b-beta/.no_exist/892b3d7a7b1cf10c7a701c60881cd93df615734c/adapter_config.json'


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
from datasets import load_dataset
from relearn.datasets.utils import (
    load_dataset as local_load_dataset,
    DATASETS_DICT,
    Datasets,
)
from relearn.datasets.corpus import process as process_corpus
from relearn.datasets.mcq import process as process_mcq

dataset_config = DATASETS_DICT[Datasets.WMDP]

# retain_dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

data_dir = Path("../data")

retain_train = local_load_dataset(data_dir, dataset_config["retain_files"])
retain_val = local_load_dataset(data_dir, dataset_config["val_retain_files"])

unlearn_files = dataset_config["unlearn_files"]
val_unlearn_files = dataset_config["val_unlearn_files"]

n_val_files = 4
max_length = 512

forget_train_1 = local_load_dataset(data_dir, unlearn_files[:n_val_files])
forget_train_2 = local_load_dataset(data_dir, unlearn_files[n_val_files:])

forget_val_1 = local_load_dataset(data_dir, val_unlearn_files[:n_val_files])
forget_val_2 = local_load_dataset(data_dir, val_unlearn_files[n_val_files:])

forget_train_1_records = process_corpus(forget_train_1, tokenizer, max_length)
forget_train_2_records = process_corpus(forget_train_2, tokenizer, max_length)
retain_train_records = process_corpus(retain_train, tokenizer, max_length)
forget_train_records = forget_train_1_records + forget_train_2_records

forget_train_mcq_1_records = process_mcq(
    forget_val_1, tokenizer, max_length, expand_choices=False
)
forget_train_mcq_2_records = process_mcq(
    forget_val_2, tokenizer, max_length, expand_choices=False
)
forget_train_mcq_records = forget_train_mcq_1_records + forget_train_mcq_2_records

forget_val_1_records = process_mcq(forget_val_1, tokenizer, max_length)
forget_val_2_records = process_mcq(forget_val_2, tokenizer, max_length)
retain_val_records = process_mcq(retain_val, tokenizer, max_length)
forget_val_records = forget_val_1_records + forget_val_2_records

Filter:   0%|          | 0/1884 [00:00<?, ? examples/s]

Map:   0%|          | 0/1884 [00:00<?, ? examples/s]

Filter:   0%|          | 0/471 [00:00<?, ? examples/s]

Map:   0%|          | 0/471 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [4]:
eval_dict = {
    "forget_1": forget_val_1_records,
    "forget_2": forget_val_2_records,
    "retain": retain_val_records,
}

In [5]:
from relearn.unlearn.rmu import train_rmu


model = train_rmu(
    model,
    forget_train_1_records,
    retain_train_records + forget_train_2_records,
    eval_records_dict=eval_dict,
    n_epochs=3,
    magnitude=6.5,
    lr=1e-5,
    forget_alpha=0.5,
    eval_at_start=False,
    max_batches=150,
)

0it [00:00, ?it/s]/mnt/align1_drive/tcqian/unlearning_order/src/relearn/unlearn/rmu/unlearn.py:122: UserWarning: Using a target size (torch.Size([1, 1, 4096])) that is different to the input size (torch.Size([4, 512, 4096])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  unlearn_loss = torch.nn.functional.mse_loss(forget_activations, control_vec) * forget_alpha
150it [00:57,  2.61it/s, Loss=0.027, Unlearn Loss=0.0249, Retain Loss=0.00209]  
100%|██████████| 314/314 [00:20<00:00, 15.19it/s]


Epoch 0 forget_1 Accuracy: 0.28343949044585987


100%|██████████| 79/79 [00:05<00:00, 14.69it/s]


Epoch 0 forget_2 Accuracy: 0.18471337579617833


100%|██████████| 393/393 [00:42<00:00,  9.18it/s]


Epoch 0 retain Accuracy: 0.5439490445859873


In [ ]:
from relearn.unlearn.rmu import train_rmu


model = train_rmu(
    model,
    forget_train_2_records,
    retain_train_records,
    eval_records_dict=eval_dict,
    n_epochs=3,
    magnitude=6.5,
    lr=5e-5,
    forget_alpha=1,
    eval_at_start=False,
    max_batches=150,
)

In [ ]:
text = "Who is the 44th president of the United States?"

# Generate text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
output = model.generate(
    **inputs,
    do_sample=True,
    max_length=100,
    num_return_sequences=5,
    temperature=0.7,
)

for i, o in enumerate(output):
    print(f"Output {i+1}: {tokenizer.decode(o, skip_special_tokens=True)}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Output 1: Who is the 44th president of the United States?

Barack Obama is the 44th president of the United States. He was born in Hawaii on August 4, 1961. His father was from Kenya and his mother was from Kansas. Obama grew up in Hawaii and later moved to Indonesia and then to Los Angeles, where he graduated from high school. He earned a bachelor's degree from Columbia University and a law degree from Harvard Law
Output 2: Who is the 44th president of the United States?

Barack Obama is the 44th president of the United States. He was inaugurated on January 20, 2009.

When is President Barack Obama's birthday?

President Barack Obama was born on August 4, 1961, in Honolulu, Hawaii.

When did President Barack Obama get married?

Pres
Output 3: Who is the 44th president of the United States?

The 44th president of the United States is Barack Obama. He served two terms in office, from 2009 to 2017. Prior to his presidency, Obama served as a state senator in Illinois and a U.S. Senator fr